In [ ]:
!pip -q install langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.0 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.252
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# RCI Chain with ChatModel

## Multi Chain

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.schema.output_parser import StrOutputParser

In [ ]:
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
    )

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me an intersting fact about {subject}"
    )

reverse_prompt = ChatPromptTemplate.from_template(
    "based on this interesting fact which is chunked down from a meta subject:\n\n {interesting_fact}\n\n Recover what the meta subject is\n Subject:"
    )

In [ ]:
chain = prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"subject": "Elvis"})

'One interesting fact about Elvis Presley is that he was a black belt in karate. He began studying martial arts in the 1950s and eventually earned his black belt in 1960. Elvis was passionate about karate and even incorporated some of the moves into his stage performances. He often practiced martial arts as a way to stay fit and maintain discipline in his life.'

In [ ]:
# full chain doesn't work
full_chain = prompt | model | StrOutputParser() | reverse_prompt | model | StrOutputParser()

In [ ]:
import langchain

langchain.debug = True

In [ ]:
full_chain.invoke({"subject": "Elvis"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "Elvis"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Chain run with input:
{
  "subject": "Elvis"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] [1.065ms] Exiting Chain run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me an intersting fact about Elvis"
  ]
}
[llm/end] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] [2.84s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "One interesting fact about Elvis Presley is that he was a black belt in karate. He began studying martial arts in the 1950s and eventually earned his black belt in 1960. Elvis was passionate about karate and even incorporated some of the moves into his performances. He often practiced with his friends and bodyguards, and even had a custom-made kar

TypeError: ignored

In [ ]:
chain1 = prompt | model | StrOutputParser()

chain2 = {"interesting_fact": chain1} | reverse_prompt | model | StrOutputParser()


In [ ]:
chain2.invoke({"subject": "elvis"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "elvis"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "subject": "elvis"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "elvis"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Chain run with input:
{
  "subject": "elvis"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] [1.426ms] Exiting Chain run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 5:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me an intersting fact about elvis"
  ]
}
[llm/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:c

"Elvis Presley's interest and proficiency in karate."

# As LCEL

In [ ]:
langchain.debug = False

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


In [ ]:
template="You are a helpful assistant that imparts wisdom and guides people with accurate answers."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
chain1 = chat_prompt | model | StrOutputParser()

In [ ]:
initial_question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"

In [ ]:
initial_answer = chain1.invoke({"question": initial_question})
initial_answer

'Roger initially has 5 tennis balls. He buys 2 cans of tennis balls, and each can has 3 tennis balls. So, the total number of tennis balls he buys is 2 cans * 3 tennis balls/can = 6 tennis balls. \n\nTherefore, Roger now has a total of 5 tennis balls + 6 tennis balls = 11 tennis balls.'

In [ ]:
fake_initial_ai_answer = """Roger initially has 5 tennis balls. Each can of tennis balls contains 3 tennis balls, and he bought 2 cans, so he has 2 x 3 = 6 additional tennis balls.
Therefore, the total number of tennis balls Roger has now is 5 + 4 = 9."""

## Part 2 - Critique  

In [ ]:
template="You are a helpful assistant that looks at answers and finds what is wrong with them based on the original question given."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="### Question:\n\n{question}\n\n ###Answer Given:{initial_answer}\n\n Review your previous answer and find problems with your answer"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
rc_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
chain2 = rc_prompt | model | StrOutputParser()

In [ ]:
constructive_criticism = chain2.invoke({"question": initial_question, "initial_answer":fake_initial_ai_answer})
constructive_criticism

'The problem with the given answer is that it incorrectly calculates the total number of tennis balls Roger has now. The answer states that Roger has 5 tennis balls initially, and then adds 4 (2 x 3) additional tennis balls from the cans he bought. However, the correct calculation should be 5 + (2 x 3) = 11. Therefore, the correct answer is that Roger has 11 tennis balls now.'

## Part 3 - The Improvement

In [ ]:
template="You are a helpful assistant that reviews answers and critiques based on the original question given and write a new improved final answer."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="### Question:\n\n{question}\n\n ###Answer Given:{initial_answer}\n\n \
###Constructive Criticism:{constructive_criticism}\n\n Based on the problems you found, improve your answer.\n\n### Final Answer:"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
improvement_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
chain3 = improvement_prompt | model | StrOutputParser()

In [ ]:
final_result = chain3.invoke({"question": initial_question,
                              "initial_answer":fake_initial_ai_answer,
                              "constructive_criticism": constructive_criticism})

final_result

'Roger initially has 5 tennis balls. He buys 2 cans of tennis balls, and each can contains 3 tennis balls. Therefore, he has 2 x 3 = 6 additional tennis balls. Adding this to the initial 5 tennis balls, Roger now has a total of 5 + 6 = 11 tennis balls.'

## Combined Chain

In [ ]:
from operator import itemgetter

In [ ]:

chain1 = chat_prompt | model | StrOutputParser()

critque_chain = {"question": itemgetter("question"),
                 "initial_answer": chain1 } | rc_prompt | model | StrOutputParser()

chain3 = {"question": itemgetter("question"),
          "initial_answer": chain1,
          "constructive_criticism": critque_chain} | improvement_prompt | model | StrOutputParser()

In [ ]:
chain3.invoke({"question":"Write an sms message to say I am tired"})

"Hey [Recipient's Name], just wanted to shoot you a quick message to let you know that I'm feeling exhausted today. I could really use some rest and relaxation to recharge. Take care!"

In [ ]:
langchain.debug = True

In [ ]:
chain3.invoke({"question":"Write an sms message to say I am tired"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "question": "Write an sms message to say I am tired"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}

[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableLambda] [4.859ms] Exiting Chain run with output:
{


"Hey [Recipient's Name], just wanted to shoot you a quick message to let you know that I am absolutely exhausted right now. I've been running on fumes and desperately need some rest and relaxation. Take care and catch up with you soon!"